# 🚀 Welcome to Your ADK Adventure - Tools & Memory! 🚀

Part 0: Setup & Authentication 🔑

In [2]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [3]:
api_key = getpass('Enter your Google API Key: ')

genai.configure(api_key=api_key)

os.environ['GOOGLE_API_KEY'] = api_key

genai.configure(api_key=api_key)
print("✅ API Key configured successfully! Let the fun begin.")

Enter your Google API Key: ··········
✅ API Key configured successfully! Let the fun begin.


In [5]:
# -- Agent Definition

# This function creates a new AI agent called "weekend_trip_agent"
def create_weekend_trip_agent():
  """Create the Spontaneous Day Trip Generator agent"""
  return Agent(
      name="weekend_trip_agent", # Internal name of the agent
      model="gemini-2.5-flash",  #The LLM powering the agent (fast + cheap)
      description="Agent specialized in generating spontaneous full-weekend itineraries based on mood, interests, and budget.",
      instruction="""
        You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

        Your Mission:
        Transform a simple mood or interest into a complete weekend-trip adventure with real-time details, while respecting a budget.

        Guidelines:
        1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
        2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
        3. **Real-Time Focus**: Search for current operating hours and special events.
        4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
       """,
      tools=[google_search]
  )
weekend_trip_agent = create_weekend_trip_agent()
print(f"🧞 Agent '{weekend_trip_agent.name}' is created and ready for adventure!")

🧞 Agent 'weekend_trip_agent' is created and ready for adventure!


In [10]:
# --- A Helper Function to Run Our Agents ---
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.

session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"


🧠 Explanation: What does this function do?

This function is responsible for running a user query through a specific agent in a session and then showing the agent's final response nicely formatted in Markdown. It helps us see not only the final answer, but also how the agent "thinks" step by step.

🔑 Key Concepts:

🏃‍♀️ Runner
The Runner is the engine that manages the conversation between the user and the agent. It handles the entire flow: sending the question, tracking the steps the agent takes (called events), and collecting the final answer.

🔄 Async Event Loop
The runner.run_async(...) function returns a stream of asynchronous events.
Each event represents something the agent is doing — like thinking, calling a tool, or responding.
The line async for event in runner.run_async(...) loops through each of these events in real time, as they happen.

This lets us follow the agent’s reasoning step by step.
Think of it like watching the agent "work things out" before answering.

👀 is_router flag
This optional flag controls whether we want to print the internal steps (events) of the agent.

When is_router = False (default), we print each event — useful when learning or debugging.

When is_router = True, we hide those steps. This is useful when an agent is being used as a tool by another agent (like a sub-agent), and we don’t want to clutter the logs.

✅ Final Output
The agent eventually sends a special event called a final response.
When that happens, we extract just the final message and display it using Markdown (to look nice in notebooks).

🧪 Why is this useful?
This pattern is great for testing and learning how agents behave.
It gives us insight into the "thinking process" of the model, not just the end result.

In [11]:
# --- Let's test the Day Trip Genie! ---

async def run_day_trip_genie():
    # Create a new, single-use session for this query
    day_trip_session = await session_service.create_session(
        app_name=weekend_trip_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Plan a relaxing and interesting weekend trip in Venecia, Italy. Keep it affordable!"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(weekend_trip_agent, query, day_trip_session, my_user_id)

await run_day_trip_genie()

🗣️ User Query: 'Plan a relaxing and interesting weekend trip in Venecia, Italy. Keep it affordable!'

🚀 Running query for agent: 'weekend_trip_agent' in session: '319c9683-b13b-4d4a-81aa-ca8a2f539bf7'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Here is a relaxing and interesting, yet affordable, spontaneous day trip itinerary for Venice, Italy:

## Spontaneous Day Trip to Venice: A Relaxing & Affordable Exploration

This itinerary focuses on experiencing the charm of Venice at a leisurely pace, exploring iconic sights without breaking the bank, and delving into quieter, more authentic neighborhoods. Wear comfortable shoes, as walking is your primary (and cheapest) mode of transport!

---

### Morning (9:00 AM - 1:00 PM): Iconic Sights & Local Buzz

*   **9:00 AM - Piazza San Marco & Basilica Exterior:** Start your day early at the iconic **Piazza San Marco (St. Mark's Square)** before the major crowds arrive. Take in the grandeur of the square, admire the exterior of *

Here is a relaxing and interesting, yet affordable, spontaneous day trip itinerary for Venice, Italy:

## Spontaneous Day Trip to Venice: A Relaxing & Affordable Exploration

This itinerary focuses on experiencing the charm of Venice at a leisurely pace, exploring iconic sights without breaking the bank, and delving into quieter, more authentic neighborhoods. Wear comfortable shoes, as walking is your primary (and cheapest) mode of transport!

---

### Morning (9:00 AM - 1:00 PM): Iconic Sights & Local Buzz

*   **9:00 AM - Piazza San Marco & Basilica Exterior:** Start your day early at the iconic **Piazza San Marco (St. Mark's Square)** before the major crowds arrive. Take in the grandeur of the square, admire the exterior of **St. Mark's Basilica**, the Doge's Palace, and the Campanile. While entry to the Basilica is generally free for visitors, arriving early can help reduce potential wait times. For prayer or mass, entry is always free via the Porta dei Fiori. *Cost: Free (exterior viewing)*.
*   **10:30 AM - Bridge of Sighs & Riva degli Schiavoni:** From St. Mark's Square, take a short stroll to glimpse the famous **Bridge of Sighs (Ponte dei Sospiri)**, connecting the Doge's Palace to the former prison. Then, promenade along the **Riva degli Schiavoni**, a beautiful waterfront promenade offering views of the lagoon and San Giorgio Maggiore island. *Cost: Free*.
*   **11:30 AM - Rialto Bridge & Market Exploration:** Walk towards the **Rialto Bridge (Ponte di Rialto)**, one of Venice's most famous landmarks. Cross the bridge and immerse yourself in the vibrant atmosphere of the **Rialto Market**. Browse the stalls selling fresh produce, fish, and local goods – it's a feast for the senses and offers a genuine taste of Venetian culture. *Cost: Free to browse*.

### Lunch (1:00 PM - 2:00 PM): Authentic & Affordable Bites

*   **1:00 PM - Cicchetti Experience:** Head to a local "bacaro" (traditional Venetian bar) for an affordable and authentic lunch experience. **Cicchetti** are small, inexpensive finger foods, similar to Spanish tapas, typically costing €1-€3 each. Pair them with a glass of local wine or a spritz. Recommended spots near Rialto or in quieter areas include **Cantina Do Spade** or **All'Arco** for a variety of delicious options, or **Bacareto da Lele** for a very local, standing-room-only experience. *Cost: ~€10-€15 for a satisfying meal*.

### Afternoon (2:00 PM - 6:00 PM): Dorsoduro's Charms & Hidden Gems

*   **2:00 PM - Dorsoduro Wandering & Basilica di Santa Maria della Salute:** Cross the Accademia Bridge (Ponte dell'Accademia) into the **Dorsoduro sestiere**. This neighborhood is known for being less crowded and offering a more authentic Venetian feel. Wander through its charming streets and make your way to the magnificent **Basilica di Santa Maria della Salute**. This stunning Baroque church, designed by Baldassare Longhena, is free to enter and explore its interior. *Cost: Free*.
*   **3:30 PM - Squero San Trovaso & Artisanal Masks:** Continue your stroll through Dorsoduro to **Squero San Trovaso**, one of the last remaining traditional gondola repair yards. You can observe gondolas being built and repaired from across the canal. Afterwards, seek out **Ca' Macana**, one of the oldest and finest mask-making workshops, where you can browse authentic Venetian masks for free. *Cost: Free*.
*   **4:30 PM - Libreria Acqua Alta:** Discover the unique and quirky **Libreria Acqua Alta**, a bookstore famous for housing its books in bathtubs, gondolas, and other waterproof containers to protect them from Venice's acqua alta (high water). It's a truly memorable and photogenic stop. *Cost: Free*.

### Evening (6:00 PM onwards): Relaxed Dinner & Evening Stroll

*   **6:00 PM - Casual Dinner in Cannaregio:** Head to the **Cannaregio neighborhood**, another area known for being less touristy and offering more local eateries. Look for a "trattoria" or "osteria" offering a "Menu del Giorno" (daily set menu) for good value. Alternatively, continue with more cicchetti at another bacaro for a light, affordable dinner. For quick and cheap options, consider **pizza al taglio** at places like Farini or fresh pasta to-go from places like **Bepe Bigoi**. *Cost: ~€15-€25, depending on choice*.
*   **8:00 PM onwards - Evening Passeggiata:** End your day with a relaxing "passeggiata" (evening stroll). Venice takes on a magical, quieter atmosphere after dark. Wander through the illuminated calli (narrow streets) and campi (squares). Enjoy the reflections of lights on the canals and simply soak in the romantic ambiance of the city. Consider walking back towards San Mark's Square for a final view without the daytime crowds. *Cost: Free*.

---

**Transportation Note:** For this itinerary, walking is highly encouraged as it's the most affordable and immersive way to experience Venice. A single vaporetto ticket costs €9.50. If you anticipate making more than 2-3 vaporetto trips in one day (e.g., if you choose to visit an outer island like San Giorgio Maggiore or explore more extensively by water), a 24-hour vaporetto pass for €25 would be more economical. Otherwise, enjoy the free and fascinating maze of Venetian streets!

--------------------------------------------------

